# Using Climada on the Euler Cluster (ETH internal)


## Access to Euler

See https://scicomp.ethz.ch/wiki/Getting_started_with_clusters for details on how to register at and get started with Euler.

For all steps below, first enter the Cluster via SSH.


## Installation and working directories

Please, get familiar with the various Euler storage options:  https://scicomp.ethz.ch/wiki/Storage_systems.<br>
As a general rule: use `/cluster/project` for installation and `/cluster/work` for data processing.

For ETH WCR group members, the suggested installation and working directories are `/cluster/project/climate/$USER` and `/cluster/work/climate/$USER` respectively.<br>
You may have to create the installation directory:

```bash
mkdir -p /cluster/project/climate/$USER \
         /cluster/work/climate/$USER
```


## Pre-installed version of Climada

Climada is pre-installed and available in the default pip environment of Euler.


### 1. Load dependencies

Use the new software stack. Unless you have already done so, run `set_software_stack.sh new`.

```bash
module load gcc/8.2.0 python/3.10.4 hdf5/1.10.1 gdal/3.4.3 geos/3.9.1 proj/8.2.1 libspatialindex/1.9.3 netcdf/4.6.3 eccodes/2.31.0 zlib/1.2.9 libtiff/4.2.0 sqlite/3.35.5
```

You need to execute this every time you login to Euler before Climada can be used. 
To safe yourself from doing it manually, append these lines to the ~/.bashrc script, which is automatically executed upon logging in to Euler.


### 2. Check installation 

```bash
python -c 'import climada; print(climada.__file__)'
```

should output something like this:

```bash
/cluster/apps/nss/gcc-8.2.0/python/3.10.4/x86_64/lib64/python3.10/site-packages/climada/__init__.py
```


### 3. Adjust the Climada configuration

Edit a configuration file according to your needs (see [Guide_Configuration](../guide/Guide_Configuration.ipynb)).
Create a climada.conf file e.g., in /cluster/home/$USER/.config with the following content:

```json
{
    "local_data": {
        "system": "/cluster/work/climate/USERNAME/climada/data",
        "demo": "/cluster/project/climate/USERNAME/climada/data/demo",
        "save_dir": "/cluster/work/climate/USERNAME/climada/results"
    }
}
```
(Replace USERNAME with your nethz-id.)


### 4. Run a job

Please see the docs at https://slurm.schedmd.com/ on how to use the `slurm` batch system 
and the Wiki https://scicomp.ethz.ch/wiki/Transition_from_LSF_to_Slurm for a mapping of `lsf` commands to their `slurm` equivalents.

```bash
cd /cluster/work/climate/$USER  # change to the working directory
sbatch [slurm options*] --wrap 'python climada_job_script.py'  # submit the job
```


## Working with Git branches

If the Climada version of the default installation is not according to your needs, you can install Climada from a local Git repository.


### 1. Load dependencies 

See [Load dependencies](#1.-load-dependencies) above.


### 2. Create installation environment

```bash
python -m venv --system-site-packages /cluster/project/climate/$USER/climada_venv
```


### 3. Checkout sources

```bash
cd /cluster/project/climate/$USER
git clone https://github.com/CLIMADA-project/climada_python.git
cd climada_python
git checkout develop  # i.e., your branch of interest
```


### 4. Pip install Climada

```bash
source /cluster/project/climate/$USER/climada_venv/bin/activate
pip install -e /cluster/project/climate/$USER/climada_python
```


### 5. Check installation

```bash
cd /cluster/work/climate/$USER
python -c 'import climada; print(climada.__file__)'
```

should output exactly this (with explicit \$USER):
```bash
/cluster/project/climate/$USER/climada_python/climada/__init__.py
```


### 6. Adjust the Climada configuration

See [Adjust the Climada configuration](#3.-adjust-the-climada-configuration) above.


### 7. Run a job

See [Run a job](#4.-run-a-job) above.


## Fallback: Conda

If Climada cannot be installed through pip because of changed dependency requirements, there is still the possibility to install Climada through the Conda environment.
> **WARNING:** This approach is highly discouraged, as it imposes a heavy and mostly unnecessary burden on the file system of the cluster.

### Installation

#### 1. Conda

Download or update to the latest version of [Miniconda](https://conda.io/miniconda.html).<br>
Installation is done by execution of the following steps:

```bash
cd /cluster/project/climate/USERNAME
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
bash Miniconda3-latest-Linux-x86_64.sh
miniconda3/bin/conda init
rm Miniconda3-latest-Linux-x86_64.sh
```

During the installation process of Miniconda, you are prompted to set the working directory according to your choice.
Set it to `/cluster/project/climate/USERNAME/miniconda3`.<br>
Once the installation has finished, log out of Euler and in again. The command prompt should be preceded by `(base)`, 
indicating that the installation was a success and that you login in into conda's base environment by default.

#### 2. Checkout sources 

See [Checkout sources](#3.-Checkout-sources) above.

#### 3. Climada environment

Create the conda environment:

```bash
cd /cluster/project/climate/USERNAME/climada_python
conda env create -f requirements/env_climada.yml --name climada_env
conda env update -n climada_env -f requirements/env_developer.yml

conda activate climada_env
conda install conda-build
conda develop .
```

#### 4. Adjust the Climada configuration

See [Adjust the Climada configuration](#3.-Adjust-the-Climada-configuration) above.

#### 5. Climada scripts 

Create a bash script for executing python scripts in the climada environment, `climadajob.sh`:
```bash
#!/bin/bash
PYTHON_SCRIPT=$1
shift
. ~/.bashrc
conda activate climada_env
python $PYTHON_SCRIPT $@
echo $PYTHON_SCRIPT completed
```

Make it executable:

```bash
chmod +x climadajob.sh
```

Create a python script that executes climada code, e.g., `climada_smoke_test.py`:

```python
import sys
from climada import CONFIG, SYSTEM_DIR
from climada.util.test.test_finance import TestNetpresValue
TestNetpresValue().test_net_pres_val_pass()
print(SYSTEM_DIR)
print(CONFIG.local_data.save_dir.str())
print("the script ran with arguments", sys.argv)
```

#### 6. Run a job 

With the scripts from above you can submit the python script as a job like this:

```bash
sbatch [slurm options] --wrap "/path/to/climadajob.sh /path/to/climada_smoke_test.py arg1 arg2"
```

After the job has finished the slurm output file should look something like this:

```
/cluster/work/climate/USERNAME/climada/data
/cluster/work/climate/USERNAME/climada/results
the script ran with arguments ['/path/to/climada_smoke_test.py', 'arg1' 'arg2']
python_script.sh completed
```

Please see the docs at https://slurm.schedmd.com/ on how to use the `slurm` batch system 
and the Wiki https://scicomp.ethz.ch/wiki/Transition_from_LSF_to_Slurm for a mapping of `lsf` commands to their `slurm` equivalents.

### Deinstallation

#### 1. Conda 

Remove the miniconda3 directory from the installation directory:

```bash
rm -rf /cluster/project/climate/USERNAME/miniconda3/
```

Delete the conda related parts from `/cluster/home/USERNAME/.bashrc`, i.e., everything between

`# >>> conda initialize >>>`\
and\
`# <<< conda initialize <<<`

#### 2. Climada

Remove the climada sources and config file:

```bash
rm -rf /cluster/project/climate/USERNAME/climada_python
rm -f /cluster/home/USERNAME/climada.conf /cluster/home/USERNAME/*/climada.conf
```


## Run Jupyter Notebook on Euler

It is possible to run a Jupyter Notebook on Euler within a JupytherHub instance running as an interactive slurm job.
See the documentation https://scicomp.ethz.ch/wiki/JupyterHub.

For using climada inside the jupyter notebook, create a `.jupyterlabrc` file in your Euler home directory with the following content:

```
module purge
module load StdEnv gcc/8.2.0 python_gpu/3.10.4 eth_proxy r/4.2.2 julia/1.8.5 hdf5/1.10.1 gdal/3.4.3 geos/3.9.1 proj/8.2.1 libspatialindex/1.9.3 netcdf/4.6.3 eccodes/2.31.0 zlib/1.2.9 libtiff/4.2.0 sqlite/3.35.5
```

Then login to https://jupyter.euler.hpc.ethz.ch/ and start a JupyterLab server.


### Using a virtual environment in a Jupyter notebook

By default the pre-installed climada version is running in your notebooks. If you want to use climada from source you can simply install a python kernel from the `climada_venv` environment, see [Working with Git branches](#working-with-git-branches)

Install an IPyhton-kernel:

```bash
source /cluster/project/climate/$USER/climada_venv/bin/activate
python -m ipykernel install --user --name climada_venv
```

Start a new JupyterLab server, the `climada_venv` kernel should appear in the list of available kernels in JupyterHub.


## Trouble shooting

### 1. Python Module not found or available

- Make sure your python environment is activated.
- Run `pip install --upgrade MISSING_MODULE`.

### 2. Upgrading from Python 3.9 to 3.10

Virtual environments created with Python 3.9 are i.g. not working for Python 3.10.
In particular Python kernels from 3.9 environments will fail to connect in a Jupyter notebook on  https://jupyter.euler.hpc.ethz.ch/.

- It's suggested to create new environments and remove the old kernels from `~/.local/share/jupyter/kernels/`.